In [2]:
import pypsa
import pandas as pd

In [3]:
n = pypsa.Network("networks/Sample/2019_181_90m_c1.25_Co2L0.0-1H.nc")

INFO:pypsa.io:Imported network 2019_181_90m_c1.25_Co2L0.0-1H.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units


In [22]:
n.carriers
n.generators.carrier
#n.generators_t.p[n.generators[n.generators.carrier.str.contains("solar")].index].sum().sum()
# Sum the power output for solar links
#n.links_t.p1[solar_links]
#n.links.carrier
#n.links_t.p1
#n.lines
#n.model definieren, rhs veränderliche sachen, lhs nicht
n.global_constraints

,type,carrier_attribute,sense,constant,investment_period,mu
GlobalConstraint,,,,,,
CO2Limit,primary_energy,co2_emissions,<=,0.000000e+00,NaN,0.0
lc_limit,transmission_expansion_cost_limit,"AC, DC",<=,1.841591e+10,NaN,0.0


## Method 1: Global constraint

In [ ]:
# Calculate total load in the system across all snapshots
total_load = n.loads_t.p_set.sum().sum()

# Define the fraction of load that should be met by wind generation
wind_fraction = 0.6  # For example, 60% from wind

# Calculate total wind generation by summing the generation from onwind, offwind-dc, and offwind-ac
total_wind_generation = (n.generators_t.p.loc[:, n.generators.carrier.isin(['onwind', 'offwind-dc', 'offwind-ac'])].sum().sum())

# Add a global constraint to enforce that wind generation covers at least 60% of the total load
n.add("GlobalConstraint",
            "wind_generation_constraint",
            type="generation_share_per_carrier",
            carrier="wind",  # Aggregated wind generation
            sense=">=",
            constant=wind_fraction * total_load)

In [23]:
n.global_constraints.loc["renewable_share"] = [
    "renewable_share",  # type: The name/type of the constraint
    "solar",            # carrier_attribute: The carrier, in this case "solar"
    ">=",               # sense: The condition (greater than or equal to)
    0.50,               # constant: The value of the constraint (e.g., 50%)
    None,               # investment_period: Set to None if not applicable
    None                # mu: Dual variable (can be None if not used)
]

## Method 2: Custom global constraint

In [ ]:
# Assuming `network` is your PyPSA network and has already been defined

# Define the fraction of load that should be met by wind generation
wind_fraction = 0.6  # 60% of load should be met by wind

# Calculate total load in the system across all snapshots
total_load = network.loads_t.p_set.sum().sum()

# Function to add a custom constraint for wind generation share
def add_wind_generation_constraint(network, snapshots):
    # Calculate total wind generation by summing onwind, offwind-dc, and offwind-ac generators
    wind_generators = network.generators.index[network.generators.carrier.isin(['onwind', 'offwind-dc', 'offwind-ac'])]
    
    # Sum the generation for wind generators
    wind_generation = network.model['Generator-p'].loc[snapshots, wind_generators].sum().sum()

    # Add the constraint: wind generation >= wind_fraction * total load
    total_load_value = total_load  # Use the total load calculated earlier
    lhs = wind_generation
    rhs = wind_fraction * total_load_value
    
    # Add the constraint to the model
    network.model.add_constraints(lhs >= rhs, name="wind_generation_constraint")

# Solve the network with the custom constraint added via `extra_functionality`
network.lopf(network.snapshots, solver_name='gurobi', extra_functionality=add_wind_generation_constraint)

## Method 3: Limiting capacities

In [9]:
# Assuming `network` is your PyPSA network

# Define the desired fractions for wind and solar
wind_fraction = 0.6  # 60% of load should be met by wind
solar_fraction = 0.3  # 30% of load should be met by solar

# Calculate the total load over all snapshots
total_load = n.loads_t.p_set.sum().sum()

# Access the capacity factors from the network (generators_t.p_max_pu)
capacity_factors = n.generators_t.p_max_pu

# Filter the capacity factors for wind and solar generators
wind_generators = n.generators.index[n.generators.carrier.isin(['onwind', 'offwind-dc', 'offwind-ac'])]
solar_generators = n.generators.index[n.generators.carrier == 'solar']

# Calculate the average capacity factors for wind and solar generators over all snapshots
avg_wind_capacity_factor = capacity_factors[wind_generators].mean().mean()
avg_solar_capacity_factor = capacity_factors[solar_generators].mean().mean()

# Calculate the minimum installed capacity required for wind and solar
min_wind_capacity = (wind_fraction * total_load) / (avg_wind_capacity_factor * len(n.snapshots))
min_solar_capacity = (solar_fraction * total_load) / (avg_solar_capacity_factor * len(n.snapshots))

# Set p_nom_min for all wind generators
n.generators.loc[wind_generators, 'p_nom_min'] = min_wind_capacity / len(wind_generators)

# Set p_nom_min for all solar generators
n.generators.loc[solar_generators, 'p_nom_min'] = min_solar_capacity / len(solar_generators)

# Loop through the affected generators and adjust p_nom_max if needed
for gen in n.generators.index:
    if n.generators.loc[gen, 'p_nom_min'] > n.generators.loc[gen, 'p_nom_max']:
        # Set p_nom_max to infinity to allow expansion if p_nom_min is higher
        n.generators.loc[gen, 'p_nom_max'] = float('inf')

In [10]:
n.export_to_netcdf("networks/Outputs/Unsolved/2019_6030mix.nc")

INFO:pypsa.io:Exported network 2019_6030mix.nc has global_constraints, loads, carriers, storage_units, buses, lines, links, generators


<xarray.Dataset> Size: 387MB
Dimensions:                               (snapshots: 8760,
                                           investment_periods: 0,
                                           global_constraints_i: 2,
                                           loads_i: 90, loads_t_p_set_i: 90,
                                           loads_t_p_i: 90, carriers_i: 11,
                                           storage_units_i: 284,
                                           ...
                                           links_t_mu_upper_i: 47,
                                           generators_i: 692,
                                           generators_t_p_max_pu_i: 614,
                                           generators_t_p_i: 692,
                                           generators_t_mu_upper_i: 692,
                                           generators_t_mu_lower_i: 611)
Coordinates: (12/35)
  * snapshots                             (snapshots) int64 70kB 0 1 ... 8759
  * investment_periods                    (investment_periods) float64 0B 
  * global_constraints_i                  (global_constraints_i) object 16B '...
  * loads_i                               (loads_i) object 720B 'AL0 0' ... '...
  * loads_t_p_set_i                       (loads_t_p_set_i) object 720B 'AL0 ...
  * loads_t_p_i                           (loads_t_p_i) object 720B 'AL0 0' ....
    ...                                    ...
  * links_t_mu_upper_i                    (links_t_mu_upper_i) object 376B '1...
  * generators_i                          (generators_i) object 6kB 'BE0 0 bi...
  * generators_t_p_max_pu_i               (generators_t_p_max_pu_i) object 5kB ...
  * generators_t_p_i                      (generators_t_p_i) object 6kB 'BE0 ...
  * generators_t_mu_upper_i               (generators_t_mu_upper_i) object 6kB ...
  * generators_t_mu_lower_i               (generators_t_mu_lower_i) object 5kB ...
Data variables: (12/101)
    snapshots_snapshot                    (snapshots) datetime64[ns] 70kB 201...
    snapshots_objective                   (snapshots) float64 70kB 1.0 ... 1.0
    snapshots_generators                  (snapshots) float64 70kB 1.0 ... 1.0
    snapshots_stores                      (snapshots) float64 70kB 1.0 ... 1.0
    investment_periods_objective          (investment_periods) float64 0B 
    investment_periods_years              (investment_periods) float64 0B 
    ...                                    ...
    generators_control                    (generators_i) object 6kB '' '' ... ''
    generators_p_nom_opt                  (generators_i) float64 6kB 95.0 ......
    generators_t_p_max_pu                 (snapshots, generators_t_p_max_pu_i) float64 43MB ...
    generators_t_p                        (snapshots, generators_t_p_i) float64 48MB ...
    generators_t_mu_upper                 (snapshots, generators_t_mu_upper_i) float64 48MB ...
    generators_t_mu_lower                 (snapshots, generators_t_mu_lower_i) float64 43MB ...
Attributes:
    network__linearized_uc:      0
    network__multi_invest:       0
    network_name:                
    network_objective:           103720801403.40538
    network_objective_constant:  5716638764.469044
    network_pypsa_version:       0.22.0
    network_srid:                4326
    meta:                        {"version": "0.7.0", "tutorial": false, "log...

In [11]:
n2 = pypsa.Network("networks/Outputs/Unsolved/2019_6030mix.nc")

INFO:pypsa.io:Imported network 2019_6030mix.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units


In [7]:
#n2.lopf(solver_name='gurobi')
n2.optimize(solver_name='gurobi')

Index(['BE1 1 offwind-ac', 'BE1 1 offwind-dc', 'BG1 1 offwind-ac',
       'BG1 1 offwind-dc', 'DE1 0 solar', 'DE1 13 offwind-ac',
       'DE1 14 offwind-ac', 'DE1 14 offwind-dc', 'DE1 20 offwind-ac',
       'DE1 20 offwind-dc',
       ...
       'RO1 1 offwind-dc', 'SE2 0 offwind-ac', 'SE2 0 offwind-dc',
       'SE2 0 solar', 'SE2 4 offwind-dc', 'SE2 4 solar', 'SE2 5 offwind-dc',
       'SE2 5 solar', 'SE2 6 solar', 'SI1 0 offwind-ac'],
      dtype='object', name='Generator', length=106)
Index(['BE1 1 offwind-ac', 'BE1 1 offwind-dc', 'BG1 1 offwind-ac',
       'BG1 1 offwind-dc', 'DE1 0 solar', 'DE1 13 offwind-ac',
       'DE1 14 offwind-ac', 'DE1 14 offwind-dc', 'DE1 20 offwind-ac',
       'DE1 20 offwind-dc',
       ...
       'RO1 1 offwind-dc', 'SE2 0 offwind-ac', 'SE2 0 offwind-dc',
       'SE2 0 solar', 'SE2 4 offwind-dc', 'SE2 4 solar', 'SE2 5 offwind-dc',
       'SE2 5 solar', 'SE2 6 solar', 'SI1 0 offwind-ac'],
      dtype='object', name='Generator', length=106)


KeyboardInterrupt: 

## TEST

In [ ]:
import pypsa
import pandas as pd
import glob
import os
import subprocess

# Parameters for the optimization solver
kwargs = {
    "threads": 0,
    "method": 2,  # barrier
    "crossover": 0,
    "BarConvTol": 1.e-5,
    "FeasibilityTol": 1.e-4,
    "OptimalityTol": 1.e-4,
    "Seed": 123,
    "AggFill": 0,
    "PreDual": 0,
    "GURO_PAR_BARDENSETHRESH": 200
}

# Step 2: Set up output directory for solved networks
output_directory = "networks/outputs/solved"
os.makedirs(output_directory, exist_ok=True)

# Step 3: Process each network file in the "networks/sample" folder
network_files = glob.glob("networks/sample/*.nc")

# Define the fraction of load that should be met by wind generation
wind_fraction = 0.6  # 60% of load should be met by wind

# Function to add a custom constraint for wind generation share
def add_wind_generation_constraint(network, snapshots):
    # Calculate total load across all snapshots
    total_load_value = network.loads_t.p_set.sum().sum()

    # Calculate total wind generation by summing onwind, offwind-dc, and offwind-ac generators
    wind_generators = network.generators.index[
        network.generators.carrier.isin(['onwind', 'offwind-dc', 'offwind-ac'])
    ]

    # Total wind generation across specified snapshots
    wind_generation = network.model['Generator-p'].loc[snapshots, wind_generators].sum(axis=1)

    # Add the constraint: wind generation >= wind_fraction * total load
    network.model.add_constraints(
        wind_generation >= wind_fraction * total_load_value,
        name="wind_generation_constraint"
    )

for network_file in network_files:
    print(f"Processing {network_file}...")

    # Load the network
    network = pypsa.Network(network_file)

    # Step 5: Solve only for the remaining snapshots with the custom constraint
    network.lopf(solver_name='gurobi', snapshots=network.snapshots[::96], extra_functionality=add_wind_generation_constraint, solver_options = kwargs)

    # Step 6: Export modified network
    output_filename = os.path.join(output_directory, os.path.basename(network_file).replace(".nc", "_MC_s.nc"))
    network.export_to_netcdf(output_filename)
    print(f"Solved and saved modified network to {output_filename}")
